In [1]:
source("../utils.R")
options(repr.plot.width = 10, repr.plot.height = 10)

Warning message in unit(c(t, r, b, l), unit):
“NAs introduced by coercion”


# Programmation orientée objet

<div class="subtitle1" id="coursename">
Techniques avancées en programmation statistique <strong>R</strong>
</div>
<div class="subtitle2" id="author">
Patrick Fournier<br>
Automne 2021<br>
Université du Québec À Montréal<br>
</div>

## Cours 4: Programmation orientée objet
1. Introduction
2. OOP et **R**
3. S3

## Introduction

* Paradigme sous lequel le concept d'*objet* joue un rôle fondamental
* Objet: peut représenter:
    + Objet physique (table, chargé de cours...)
    + Concept (matrice, paix dans le monde...)
* Approche OO:
    + Définir un ensemble d'objets
    + Définir les interactions entre ces objets
* De ce point de vue, possible dans la plupart des langages
* Premier langage OO: Simula, 1962.
    + Destiné aux simulations Monte Carlo 🧮

### Terminologie 1

* *Classe*: Déclaration (parfois définition) de la structure interne d'un ensemble d'objets
* *Slot*/*field*/*attribut*: Variable associée à un objet / une classe
* *Méthode*: Fonction associée à un ou plusieurs objet(s) / classe(s)
* *Héritage*: Processus par lequel une classe fille acquiert la structure d'une ou plusieurs classe(s) mère(s)
* *Polymorphisme*: Une interface pour plusieurs entités de types différents
* *Encapsulation*: Cacher l'implémentation à l'utilisateur

### Terminologie 2
* *Message passing*: objets $ \Rightarrow $ communiquent entre eux par messages
    + *Message* $ = $ idée fondamentale de l'OOP originale (Alan Kay & Smalltalk)
    + Souvent limités à des appels de méthodes (C++, Java...)
    + Concept beaucoup plus large (Smalltalk, Groovy...)
* *Fonction générique*: appels de méthodes $ \Rightarrow $ *dispatchés* par des *fonctions génériques*
    + Origines: Flavors (Lisp Machine Lisp) & CommonLoops (Common Lisp)
    + Approche "de base" de **R**

## OOP et **R**
### Systèmes d'objets

* *S3* ([Advanced R](https://adv-r.hadley.nz/s3.html))
    + Fonctions génériques
    + Pas (vraiment) de classe
* *S4* ([Advanced R](https://adv-r.hadley.nz/s4.html))
    + S3 $ + $ classes formelles
* *Reference classes* (*RC*, *R5*)
    + Message passing
    + Instances mutables
* *R6* ([Advanced R](https://adv-r.hadley.nz/r6.html), [Site officiel](https://r6.r-lib.org))
    + "Version améliorée" de RC
* *Closure*

### Choisir un système
* S3
    + Très informel
    + Peu de fonctionnalités
    + Facile à apprendre
    + Facile à utiliser
    + Suffisant pour un grand nombre d'utilisations
* S4
    + Moins performant
    + Plus difficile à utiliser
* RC
    + Moins performant que R6
    + Objets mutables
    + Basé sur S4
* R6
    + Basé sur S3
    + Essentiellement RC, mais "mieux" 

### Choisir un système

* Choix par défaut: S3
* Si la mutabilité est importante: R6
* Utilisez S4 si vous avez besoin
    + Héritage multiple (plus d'une classe mère)
    + Dispatch multiple (dispatch sur plusieurs arguments)
    + Interaction avec [Bioconductor](https://www.bioconductor.org/)
* Utilisez RC si vous êtes obligés!

[Détails](https://adv-r.hadley.nz/oo-tradeoffs.html)

## S3
### Attribut `class`

* S3 est basé sur une seule chose: l'*attribut class*
* Vecteur de chaine de caractères (`character`)
* Entrées ordonnées de la classe la plus spécifique à la moins spécifique

### S3: structure

* Un objet S3 est composé de
    + Un objet "standard"
    + Un attribut `class`
* Objet standard: souvent une `list`, mais aucune obligation

In [2]:
p1 <- c(1, 2)
class(p1) <- c("point", "numeric")

In [3]:
p2 <- c(1, 2)
attributes(p2) <- list(class = c("point", "numeric"))

In [4]:
p3 <- structure(c(3, 4), class = c("point", "numeric"))

### S3: structure

* $ \Rightarrow $ Possible de changer la classe d'un objet a posteriori
* Déconseillé, surtout si l'objet a été créé par quelqu'un d'autre

### Constructeur

* L'appel à `class<-`, `attributes<-` ou `structure` n'est pas élégant, pratique ni sécuritaire
* $ \Rightarrow $ Il est d'usage de définir un (plusieurs) constructeur(s) 👷
* Simple fonction qui se charge de l'appel
    + Après avoir vérifié le type des arguments!
* Habituellement, nom constructeur $ = $ nom classe

In [5]:
point <- function(v){
    v <- as.numeric(v)
    
    stopifnot(identical(length(v), 2L))

    structure(v, class = c("point", class(v)))
}

In [6]:
point(1:3)

ERROR: Error in point(1:3): identical(length(v), 2L) is not TRUE


In [7]:
point(letters[1:2])

Warning message in point(letters[1:2]):
“NAs introduced by coercion”


[1] NA NA
attr(,"class")
[1] "point"   "numeric"

In [8]:
point(1:2)

[1] 1 2
attr(,"class")
[1] "point"   "numeric"

### Méthodes

* Reconaissables par leur nom: `méthode.classe(...)`
* $ \Rightarrow $ facile de définir de nouvelles méthodes!
    + Suffisant de définir une fonction nommée de manière appropriée
* Possible d'implémenter des méthodes pour une classe que l'on a pas définie
    + `Monkey patching`
    + À utiliser avec parcimonie!

### Méthodes

* Lorsque le nom d'une variable est saisi, **R** appelle `print`sur cette variable
* Nous allons:
    1. Définir une méthode `print`pour notre type `point`
    2. Définir une méthode `summary` comme une version détaillée de `print`
* `Important`: Arguments méthode $ = $ arguments générique!

In [9]:
formals(print)

$x


$...



In [10]:
print.point <- function(x, ...)
    cat("x = ", x[1], "& y = ", x[2], "\n")

In [11]:
summary.point <- function(object, ...){
    cat("Point de coordonnées ")
    print(object)
}

In [12]:
p1

x =  1 & y =  2

In [13]:
summary(p1)

Point de coordonnées x =  1 & y =  2 


In [14]:
summary(cars)

     speed           dist       
 Min.   : 4.0   Min.   :  2.00  
 1st Qu.:12.0   1st Qu.: 26.00  
 Median :15.0   Median : 36.00  
 Mean   :15.4   Mean   : 42.98  
 3rd Qu.:19.0   3rd Qu.: 56.00  
 Max.   :25.0   Max.   :120.00  

In [15]:
summary(lm(dist ~ speed, cars))


Call:
lm(formula = dist ~ speed, data = cars)

Residuals:
    Min      1Q  Median      3Q     Max 
-29.069  -9.525  -2.272   9.215  43.201 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -17.5791     6.7584  -2.601   0.0123 *  
speed         3.9324     0.4155   9.464 1.49e-12 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 15.38 on 48 degrees of freedom
Multiple R-squared:  0.6511,	Adjusted R-squared:  0.6438 
F-statistic: 89.57 on 1 and 48 DF,  p-value: 1.49e-12


### Method dispatch

* Les deux expressions sont des appels à la même fonction: `summary`
* Différence: *argument* fourni à summary
    + En fait, *classe* de l'argument
* Exemple de *polymorphisme*
    + Interface commune: `summary`
    + Comportement différent en fonction de la classe de l'argument

### Method dispatch

* Fonctionnement de la fonction `summary` est très complexe
* Son code doit être remarquablement compliqué

In [16]:
body(summary)

UseMethod("summary")

<font size = 50>???</font>

### Method dispatch
* `summary` ne calcule aucune statistique sommaire et n'affiche rien directement 😱
* Rôle: déterminer la *méthode* appropriée à appeller $ \Rightarrow $ *method dispatch* 
* Fonction qui dispatch: *fonction générique*
* En R: se reconnaissent facilement par leur appel à `UseMethod`
* `UseMethod`trouve la méthode la plus *spécialisée* pouvant s'appliquer

In [24]:
v1 <- structure(c(1, 2), class = c("vec", "numeric"))

summary.vec

ERROR: Error in eval(expr, envir, enclos): object 'summary.vec' not found


In [19]:
summary.numeric

ERROR: Error in eval(expr, envir, enclos): object 'summary.numeric' not found


In [21]:
methods(summary)

 [1] summary.aov                    summary.aovlist*              
 [3] summary.aspell*                summary.check_packages_in_dir*
 [5] summary.connection             summary.data.frame            
 [7] summary.Date                   summary.default               
 [9] summary.ecdf*                  summary.factor                
[11] summary.ggplot*                summary.glm                   
[13] summary.hcl_palettes*          summary.infl*                 
[15] summary.lm                     summary.loess*                
[17] summary.manova                 summary.matrix                
[19] summary.mlm*                   summary.nls*                  
[21] summary.packageStatus*         summary.point                 
[23] summary.POSIXct                summary.POSIXlt               
[25] summary.ppr*                   summary.prcomp*               
[27] summary.princomp*              summary.proc_time             
[29] summary.rlang_error*           summary.rlang_trace*      

* Exemple: initialement, `summary` cherchait une méthode pour la classe la plus spécifique de `v1`, `vec`
    + N'existe pas $ \Rightarrow $ recommence la recherche pour la classe `numeric`
* Arrivé à ce point, **R** a épuisé toutes les classes
* Pourtant, `summary` fonctionne quand même 🤨

In [25]:
summary(v1)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   1.00    1.25    1.50    1.50    1.75    2.00 

* Dernier atout: `summary.default`
* En général, avant d'abandonner, **R** cherche une méthode par défaut
    + Utile pour définir un comportement par défaut

In [26]:
summary.default(v1)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   1.00    1.25    1.50    1.50    1.75    2.00 

### Fonction générique

* Méthode *sans* fonction générique: ☹️
* $ \Rightarrow $ Nécessité de définir nos propres génériques (parfois)
* La plupart du temps, se limite à un appel à `UseMethod` qui prend 2 arguments:
    + *generic*: nom de la méthode à appeler
    + *object*: objet dont le type est utilisé pour le dispatch.
        * Par défaut, premier argument passé à la fonction générique 🪄

### Fonction générique

Définissons une méthode permettant de déterminer si des points sont alignés

In [31]:
isaligned.point <- function(...) {
    points <- list(...)
    
    isTRUE(length(points) <= 2) && return(TRUE)
    
    fit <- crossprod(
        solve(cbind(1, c(points[[1]][1], points[[2]][1]))),
        c(points[[1]][2], points[[2]][2]))
    
    for (p in points[-(1:2)]) {
        pred <- crossprod(c(1, p[1]), fit)[1]
        
        isTRUE(all.equal(pred, p[2])) || return(FALSE)
    }
    
    TRUE
}

In [32]:
isaligned.point(point(1:2), point(3:4), point(5:6))

[1] TRUE

In [33]:
isaligned.point(point(1:2), point(3:4), point(c(5, 7)))

[1] FALSE

### Fonction générique

Définissons une générique

In [34]:
isaligned <- function(...) UseMethod("isaligned")

In [35]:
isaligned(point(1:2), point(5:6), point(9:10))

[1] TRUE

In [36]:
isaligned(point(1:2), point(5:6), point(c(9, 11)))

[1] FALSE

### Exemple

* Classe `point`: simple spécialisation de la classe `numeric`
* Peut facilement se complexifier!
* Exemple: offrir la possibilité à l'utilisateur de marquer son point à l'aide d'une chaîne de caractère

In [67]:
pointM <- function(v, mark = NULL) {
    stopifnot(is.null(mark) ||
              (is.character(mark) && identical(length(mark), 1L)))
    
    p <- point(v)
    attr(p, "mark") <- mark
    
    structure(p, class = c("pointM", class(p)))
}

### Exemple

* On crée notre nouvelle classe en ajoutant un attribut à un `point`
    + fonction `attr<-`
* Possibilité d'ajouter un nombre arbitraire d'attributs à un objet
    + Certains (ex. `class`, `dim`) jouent un rôle spécial
    + Voir documentation de `attr`
* Autre possibilité: utiliser une liste
    + Première entrée: `point`
    + Seconde entrée: marque
* Notre approche comporte certains avantages

In [41]:
p1 <- pointM(1:2, "Premier point")
p2 <- pointM(3:4, "Second point")
p3 <- pointM(5:6, "Troisième point")

Comme nos `pointM` héritent de `point`, on dispose déjà de quelques méthodes!

In [42]:
p1

x =  1 & y =  2

In [43]:
summary(p2)

Point de coordonnées x =  3 & y =  4 


In [44]:
isaligned(p1, p2, p3)

[1] TRUE

### Exemple

* Conceptuellement correct: tout ce qui peut être fait avec un point devrait pouvoir se faire avec un point marqué
* Fournissons à l'utilisateur un moyen de voir la marque d'un point

In [68]:
mark.pointM <- function(point) {
    print(attr(point, "mark"))
}

mark <- function(point) UseMethod("mark")

In [48]:
mark(p1)

[1] "Premier point"


### Exemple

Malheureusement, pas possible de modiifer la marque de la manière "habituelle"

In [49]:
mark(p1) <- "1er point"

ERROR: Error in mark(p1) <- "1er point": could not find function "mark<-"


### Exemple

Solution:

In [50]:
`mark<-.pointM` <- function(point, value) {
    attr(point, "mark") <- value
    
    point
}

`mark<-` <- function(point, value) UseMethod("mark<-")

In [51]:
mark(p1)

[1] "Premier point"


In [52]:
mark(p1) <- "1er point"

In [53]:
mark(p1)

[1] "1er point"


### Exemple

Finalement, spécialisons `print`

In [60]:
print.pointM <- function(x, ...) {
    print.point(x)
    cat(mark(x), "\n")
}

In [63]:
print(p1)

x =  1 & y =  2 
[1] "1er point"
1er point 


In [64]:
summary(p2)

Point de coordonnées x =  3 & y =  4 
[1] "Second point"
Second point 


### Performance

* Système par fonction générique $ \Rightarrow $ *grande* flexibilité
* Coût: fonction générique $ \Rightarrow $ plus d'appels de fonctions
* En général, pas un problème
    + Dans du code critique (boucle...), considérer appeler directement la méthode